# __init__.py
Before Python3.3, every package must contain a file named `__init__.py`.  
This file is run automatically the first time a Python program imports this package.  

Although after Python3.3, directory modules can be imported as without `__init__.py`. (Using namespace to do so)  
Using `__init__.py` provide a performance advantage.  



## What does it do?

### 1. Package initialization.
All the code in the directory's `__init__.py` is run automatically.  
It can be used to initialize the state  
e.g. initialize file to create required data files, open connections to databases

### 2. Module usability declarations
Declare that a directory is a Python Package  
In fact, `__init__.py` are often empty in practice

## from * statament behavior
You can use `__all__` list in `__init__.py` to define what is exported when this directory is imported using `from *` statement

```python
# c.py

x = 1
y = 2
__all__ = [x]
```

In [6]:
from c import *

print(x)
print(y)   # not imported since it's not in __all__

1


NameError: name 'y' is not defined

---

# Package Relative Imports
Python2 implicitly searches package directories first on imports while Python3 requires explicit relative import syntax in order to making same-package imports.  
This change enhances code readability by making same-package imports more obvious

---

# Namespace Package (Python3.3)


In [7]:
# ns/mod.py (without __init__.py)

import ns.mod

This is mod.py


In [8]:
ns

<module 'ns' (namespace)>

In [9]:
ns.__path__

_NamespacePath(['/Users/LeeW/Coding/learning_note/Learning_Python/Part 5 - Modules and Packages/ns'])

- Cannot contain an `__init__.py` (Since it stops the following algorithm)  
- May span multiple directories that are collected at import time  
- Relative imports work in namespace packages too  

## The import algorithm
1. If `directory/module/__init__.py` is found, a regular package is imported
2. If `directory/module.{py, pyc ...}` is found ,a simple module is imported
3. If `directory/Module` is found and is a directory, it's record and the scan continues with the next directory in the search path
4. If none of the above was found, the scan continues with the next directory in the search path.

If the search path scan completes without returning a module or package by steps 1 or 2, and at least one directory was recorded by step 3, than a namespace package is created.  

Once a namespace package is created, there is no difference between it and a regular package.  

## Optional __init__.py ?
Many packages require no initialization code.  
Thus, they are no longer required after Python3.3  

However, there is performance advantage to have one.  
With namespace packages, all entries in the path must be scanned.  
More formally, regular packages stop the algorithm at step 1.  